In [1]:
import cProfile
import lysis
import math
import os
import numpy as np
from collections import Counter

In [2]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code='2023-01-31-1301')

In [3]:
# p = {"rows": 12,
#          "cols": 9,
#          "empty_rows": 3,
#          "total_molecules": 430,
#          "total_time": 10*60,
#          "duplicate_fortran": True
#          }
# e.initialize_macro_param(p)
# e.read_file()

In [3]:
e.read_file()

In [14]:
e.to_file()

In [4]:
macro = lysis.MacroscaleRun(e)

In [6]:
r3 = np.array([0.013*100])
macro.find_unbinding_time(r3, 0.0)

array([0.37808582])

In [22]:
unbinding_time_bin = r3
count = len(r3)
current_time = 0.0
lysis_time_bin = np.array([0.0377])

lysis_time_bin = lysis_time_bin * (macro.exp.macro_params.microscale_runs / 100)
interp = np.full(count, float("inf"), dtype=np.double)
unbinding_time_bin = unbinding_time_bin.astype(int)
total_lyses = macro.exp.data.total_lyses[unbinding_time_bin] - 1
lysis_happens = lysis_time_bin < total_lyses
interp[~lysis_happens] = float("inf")

# TODO(bpaynter): Should be able to improve this with a 2D
#                 interpolation or a custom numpy kernel
for i in np.arange(count)[lysis_happens]:
    interp[i] = np.interp(
        lysis_time_bin[i],
        np.arange(total_lyses[i]),
        macro.exp.data.lysis_time[unbinding_time_bin[i], : total_lyses[i]],
    )
interp # + (current_time - macro.exp.macro_params.time_step / 2)

array([44.98754296])

In [18]:
macro.exp.macro_params.time_step

0.0003423940833333333

In [11]:
macro.exp.data.lysis_time[unbinding_time_bin[i], : total_lyses[i]]

array([26.00913947, 26.77610749, 28.00890967, 30.68915295, 30.76509679,
       30.86917166, 30.96479071, 32.08974628, 32.1114515 , 32.50776026,
       32.60855094, 32.81468819, 33.08845622, 35.00468359, 35.81625917,
       36.40427586, 36.75465857, 38.80920826, 41.61935759, 45.58192862])

In [12]:
np.arange(total_lyses[0])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [13]:
lysis_time_bin

array([19.35])

In [14]:
total_lyses

array([20])

In [15]:
unbinding_time_bin

array([1])

In [5]:
macro.run()

  0%|          | 0/1752366 [00:00<?, ?it/s]

In [6]:
macro.bind_info_file.close()

In [10]:
#n = np.empty((0,), dtype=np.float_)
with open(os.path.join(macro.exp.os_path, "m_bind_t.p-array.dat"), 'r') as file:
    n = np.fromregex(file, r'(\-?\d\.\d*e?\-?\+?\d*)\w*(\-?\d\.\d*e?\-?\+?\d*)\w*(\-?\d\.\d*e?\-?\+?\d*)', dtype=[('r1', np.float_), ('t', np.float_), ('bind', np.float_)])
n

array([], dtype=[('r1', '<f8'), ('t', '<f8'), ('bind', '<f8')])

In [19]:
np.array2string(n)[1:-1]

'9.50116062e-43 9.15604175e-72 1.39804328e-76 ... 5.06869768e-86\n 5.64341326e-62 5.39365393e-62'

In [7]:
current_time = 2920 * macro.exp.macro_params.time_step
current_time

0.9997907233333332

In [9]:
macro.m_fiber_status = macro.fiber_status[macro.location]
macro.unbind_by_degradation(macro.bound & (macro.m_fiber_status < current_time), current_time)
macro.unbind_by_time(macro.bound & (macro.binding_time < current_time), current_time)

should_bind = (~macro.bound
               & (macro.binding_time < current_time)
               & (macro.m_fiber_status > current_time)
               & (macro.waiting_time < current_time))

move_chance = np.ones(macro.exp.macro_params.total_molecules, dtype=np.double)
move_chance[~macro.bound] = macro.rng.random(np.count_nonzero(~macro.bound), dtype=np.double)
should_move = move_chance < macro.exp.macro_params.moving_probability
conflict = should_bind & should_move
threshold = np.full(macro.exp.macro_params.total_molecules, -1, dtype=np.double)
threshold[conflict] = ((current_time - macro.binding_time[conflict])
                       / macro.exp.macro_params.time_step)
should_bind[conflict] = macro.rng.random(np.count_nonzero(conflict)) > threshold[conflict]
should_move[conflict] = ~should_bind[conflict]

macro.bind(should_bind, current_time)

In [10]:
still_stuck_to_fiber = np.full(macro.exp.macro_params.total_molecules, False, dtype=np.bool_)
still_stuck_to_fiber[should_bind] = macro.waiting_time[should_bind] > current_time
np.logical_and(still_stuck_to_fiber[should_bind], macro.unbound_by_degradation[should_bind], out=still_stuck_to_fiber[should_bind])
np.count_nonzero(still_stuck_to_fiber)

0

In [11]:
for n in np.arange(macro.exp.macro_params.total_molecules)[still_stuck_to_fiber]:
    print(n)

In [15]:
a = 499
b = 500
c = a*e.macro_params.time_step
d = b*e.macro_params.time_step

In [16]:
%timeit a < b

36.1 ns ± 0.269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [17]:
%timeit c < d

33.1 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
lysis.from_fortran_edge_index(51, 4, 5)

In [3]:
rng = np.random.default_rng(seed=489489496)

In [5]:
a = rng.random(60).reshape(6, 10)

In [7]:
b = rng.random(10)
c = b < 0.5

In [8]:
c

array([ True, False, False,  True, False,  True, False,  True,  True,
        True])

In [10]:
a[5][c]

array([0.46961349, 0.51494345, 0.93896747, 0.49600301, 0.5810864 ,
       0.22792777])

In [11]:
a[5]

array([0.46961349, 0.92091926, 0.20134264, 0.51494345, 0.99789049,
       0.93896747, 0.76984013, 0.49600301, 0.5810864 , 0.22792777])

In [50]:
c

array([[False, False,  True,  True],
       [False, False,  True,  True],
       [False,  True,  True,  True]])

In [48]:
np.take_along_axis(a[b], c[:,::-1], axis=1)

array([[0.28751422, 0.17398308, 0.55931457, 0.53022389],
       [0.34155978, 0.28751422, 0.51085294, 0.95012645],
       [0.24432482, 0.24990649, 0.08039694, 0.63829939]])

In [73]:
d = np.count_nonzero(c, axis=1)
d

array([2, 2, 3], dtype=int64)

In [71]:
e = d[np.newaxis, :]
np.ix_(,d)

(array([[0]], dtype=int64), array([[2, 2, 3]], dtype=int64))

In [74]:
a[b][np.full(3, True), d]

array([0.28751422, 0.51085294, 0.63829939])

In [81]:
%timeit np.any(a < 0.1)

8.26 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [83]:
a = np.empty((20, 9), dtype=int)

In [84]:
a[:] = np.arange(9)

In [85]:
a

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [86]:
a[4,6]=100
a

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5, 100,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3, 

In [3]:
tot_sec = 6110*10
tot_sec // (60*60), (tot_sec // 60) % 60, tot_sec % 60

(16, 58, 20)

In [10]:
import dataclasses
import inspect

import numpy as np

import lysis

In [9]:
sig = inspect.signature(lysis.util.MacroParameters)

In [10]:
for key in sig.parameters:
    print(key, sig.parameters[key].default)

binding_rate 0.1
pore_size 0.00010135
diffusion_coeff 5e-07
binding_sites 427
forced_unbind 0.0852
average_bind_time 27.8
grid_node_distance 1.0862
cols 93
rows 121
empty_rows 28
total_molecules 43074
moving_probability 0.2
microscale_runs 50000
total_trials 10
total_time 1200
seed -2137354075
state (129281, 362436069, 123456789, None)
save_interval 10
macro_version diffuse_into_and_along
log_lvl 30
duplicate_fortran False
processing_library numpy


In [18]:
inspect.getdoc(e.macro_params.binding_rate)

'Convert a string or number to a floating point number, if possible.'

In [20]:
lysis.util.MacroParameters.binding_rate.__doc__

'Convert a string or number to a floating point number, if possible.'

In [6]:
a = e.macro_params.fortran_names()

In [4]:
a = lysis.util.FortranMacro(e, "../..", "bin/macro-normal", ".dat", ".f-normal.dat")

In [6]:
a.run()

['bin/macro-normal', '--expCode', '2023-01-30-1700', '--inFileCode', '.dat', '--outFileCode', '.f-normal.dat', '--N', '9', '--F', '12', '--Ffree', '2', '--M', '430', '--tf', '600']
None


In [9]:
"binding_rate" in a

True

In [8]:
a

{'binding_rate': 'kon',
 'pore_size': 'delx',
 'diffusion_coeff': 'Diff',
 'binding_sites': 'bs',
 'forced_unbind': 'frac_forced',
 'average_bind_time': 'avgwait',
 'grid_node_distance': 'dist',
 'cols': 'N',
 'rows': 'F',
 'total_edges': 'num',
 'empty_rows': 'Ffree-1',
 'last_empty_edge': 'enoFB-1',
 'total_molecules': 'M',
 'moving_probability': 'q',
 'microscale_runs': 'nummicro',
 'total_trials': 'stats',
 'total_time': 'tf',
 'time_step': 'tstep',
 'total_time_steps': 'num_t',
 'seed': 'seed',
 'state': 'state',
 'number_of_saves': 'nplt'}

In [7]:
s = "Ffree-1"
s[:-2]

'Ffree'

In [6]:
f_n = np.array([[  1,  1,  1,  1, 19, 19, 19, 19],
    [  0,  0,  2,  2, 19, 19, 20, 20],
    [  1,  1,  3,  3, 20, 20, 20, 20],
    [  2,  2,  4,  4, 20, 20, 21, 21],
    [  3,  3,  5,  5, 21, 21, 21, 21],
    [  4,  4,  6,  6, 21, 21, 22, 22],
    [  5,  5,  7,  7, 22, 22, 22, 22],
    [  6,  6,  8,  8, 22, 22, 23, 23],
    [  7,  7,  9,  9, 23, 23, 23, 23],
    [  8,  8, 10, 10, 23, 23, 24, 24],
    [  9,  9, 11, 11, 24, 24, 24, 24],
    [ 10, 10, 12, 12, 24, 24, 25, 25],
    [ 11, 11, 13, 13, 25, 25, 25, 25],
    [ 12, 12, 14, 14, 25, 25, 26, 26],
    [ 13, 13, 15, 15, 26, 26, 26, 26],
    [ 14, 14, 16, 16, 26, 26, 27, 27],
    [ 15, 15, 17, 17, 27, 27, 27, 27],
    [ 16, 16, 18, 18, 27, 27, 28, 28],
    [ 17, 17, 17, 17, 28, 28, 28, 28],
    [  0,  0,  1,  1, 29, 29, 30, 30],
    [  1,  2,  2,  3, 30, 31, 31, 32],
    [  3,  4,  4,  5, 32, 33, 33, 34],
    [  5,  6,  6,  7, 34, 35, 35, 36],
    [  7,  8,  8,  9, 36, 37, 37, 38],
    [  9, 10, 10, 11, 38, 39, 39, 40],
    [ 11, 12, 12, 13, 40, 41, 41, 42],
    [ 13, 14, 14, 15, 42, 43, 43, 44],
    [ 15, 16, 16, 17, 44, 45, 45, 46],
    [ 17, 17, 18, 18, 46, 46, 47, 47],
    [ 19, 19, 30, 30, 30, 30, 48, 48],
    [ 19, 20, 29, 29, 31, 31, 48, 49],
    [ 20, 20, 30, 30, 32, 32, 49, 49],
    [ 20, 21, 31, 31, 33, 33, 49, 50],
    [ 21, 21, 32, 32, 34, 34, 50, 50],
    [ 21, 22, 33, 33, 35, 35, 50, 51],
    [ 22, 22, 34, 34, 36, 36, 51, 51],
    [ 22, 23, 35, 35, 37, 37, 51, 52],
    [ 23, 23, 36, 36, 38, 38, 52, 52],
    [ 23, 24, 37, 37, 39, 39, 52, 53],
    [ 24, 24, 38, 38, 40, 40, 53, 53],
    [ 24, 25, 39, 39, 41, 41, 53, 54],
    [ 25, 25, 40, 40, 42, 42, 54, 54],
    [ 25, 26, 41, 41, 43, 43, 54, 55],
    [ 26, 26, 42, 42, 44, 44, 55, 55],
    [ 26, 27, 43, 43, 45, 45, 55, 56],
    [ 27, 27, 44, 44, 46, 46, 56, 56],
    [ 27, 28, 45, 45, 47, 47, 56, 57],
    [ 28, 28, 46, 46, 46, 46, 57, 57],
    [ 29, 29, 30, 30, 58, 58, 59, 59],
    [ 30, 31, 31, 32, 59, 60, 60, 61],
    [ 32, 33, 33, 34, 61, 62, 62, 63],
    [ 34, 35, 35, 36, 63, 64, 64, 65],
    [ 36, 37, 37, 38, 65, 66, 66, 67],
    [ 38, 39, 39, 40, 67, 68, 68, 69],
    [ 40, 41, 41, 42, 69, 70, 70, 71],
    [ 42, 43, 43, 44, 71, 72, 72, 73],
    [ 44, 45, 45, 46, 73, 74, 74, 75],
    [ 46, 46, 47, 47, 75, 75, 76, 76],
    [ 48, 48, 59, 59, 59, 59, 77, 77],
    [ 48, 49, 58, 58, 60, 60, 77, 78],
    [ 49, 49, 59, 59, 61, 61, 78, 78],
    [ 49, 50, 60, 60, 62, 62, 78, 79],
    [ 50, 50, 61, 61, 63, 63, 79, 79],
    [ 50, 51, 62, 62, 64, 64, 79, 80],
    [ 51, 51, 63, 63, 65, 65, 80, 80],
    [ 51, 52, 64, 64, 66, 66, 80, 81],
    [ 52, 52, 65, 65, 67, 67, 81, 81],
    [ 52, 53, 66, 66, 68, 68, 81, 82],
    [ 53, 53, 67, 67, 69, 69, 82, 82],
    [ 53, 54, 68, 68, 70, 70, 82, 83],
    [ 54, 54, 69, 69, 71, 71, 83, 83],
    [ 54, 55, 70, 70, 72, 72, 83, 84],
    [ 55, 55, 71, 71, 73, 73, 84, 84],
    [ 55, 56, 72, 72, 74, 74, 84, 85],
    [ 56, 56, 73, 73, 75, 75, 85, 85],
    [ 56, 57, 74, 74, 76, 76, 85, 86],
    [ 57, 57, 75, 75, 75, 75, 86, 86],
    [ 58, 58, 59, 59, 87, 87, 88, 88],
    [ 59, 60, 60, 61, 88, 89, 89, 90],
    [ 61, 62, 62, 63, 90, 91, 91, 92],
    [ 63, 64, 64, 65, 92, 93, 93, 94],
    [ 65, 66, 66, 67, 94, 95, 95, 96],
    [ 67, 68, 68, 69, 96, 97, 97, 98],
    [ 69, 70, 70, 71, 98, 99, 99,100],
    [ 71, 72, 72, 73,100,101,101,102],
    [ 73, 74, 74, 75,102,103,103,104],
    [ 75, 75, 76, 76,104,104,105,105],
    [ 77, 77, 88, 88, 88, 88,106,106],
    [ 77, 78, 87, 87, 89, 89,106,107],
    [ 78, 78, 88, 88, 90, 90,107,107],
    [ 78, 79, 89, 89, 91, 91,107,108],
    [ 79, 79, 90, 90, 92, 92,108,108],
    [ 79, 80, 91, 91, 93, 93,108,109],
    [ 80, 80, 92, 92, 94, 94,109,109],
    [ 80, 81, 93, 93, 95, 95,109,110],
    [ 81, 81, 94, 94, 96, 96,110,110],
    [ 81, 82, 95, 95, 97, 97,110,111],
    [ 82, 82, 96, 96, 98, 98,111,111],
    [ 82, 83, 97, 97, 99, 99,111,112],
    [ 83, 83, 98, 98,100,100,112,112],
    [ 83, 84, 99, 99,101,101,112,113],
    [ 84, 84,100,100,102,102,113,113],
    [ 84, 85,101,101,103,103,113,114],
    [ 85, 85,102,102,104,104,114,114],
    [ 85, 86,103,103,105,105,114,115],
    [ 86, 86,104,104,104,104,115,115],
    [ 87, 87, 88, 88,116,116,117,117],
    [ 88, 89, 89, 90,117,118,118,119],
    [ 90, 91, 91, 92,119,120,120,121],
    [ 92, 93, 93, 94,121,122,122,123],
    [ 94, 95, 95, 96,123,124,124,125],
    [ 96, 97, 97, 98,125,126,126,127],
    [ 98, 99, 99,100,127,128,128,129],
    [100,101,101,102,129,130,130,131],
    [102,103,103,104,131,132,132,133],
    [104,104,105,105,133,133,134,134],
    [106,106,117,117,117,117,135,135],
    [106,107,116,116,118,118,135,136],
    [107,107,117,117,119,119,136,136],
    [107,108,118,118,120,120,136,137],
    [108,108,119,119,121,121,137,137],
    [108,109,120,120,122,122,137,138],
    [109,109,121,121,123,123,138,138],
    [109,110,122,122,124,124,138,139],
    [110,110,123,123,125,125,139,139],
    [110,111,124,124,126,126,139,140],
    [111,111,125,125,127,127,140,140],
    [111,112,126,126,128,128,140,141],
    [112,112,127,127,129,129,141,141],
    [112,113,128,128,130,130,141,142],
    [113,113,129,129,131,131,142,142],
    [113,114,130,130,132,132,142,143],
    [114,114,131,131,133,133,143,143],
    [114,115,132,132,134,134,143,144],
    [115,115,133,133,133,133,144,144],
    [116,116,117,117,145,145,146,146],
    [117,118,118,119,146,147,147,148],
    [119,120,120,121,148,149,149,150],
    [121,122,122,123,150,151,151,152],
    [123,124,124,125,152,153,153,154],
    [125,126,126,127,154,155,155,156],
    [127,128,128,129,156,157,157,158],
    [129,130,130,131,158,159,159,160],
    [131,132,132,133,160,161,161,162],
    [133,133,134,134,162,162,163,163],
    [135,135,146,146,146,146,164,164],
    [135,136,145,145,147,147,164,165],
    [136,136,146,146,148,148,165,165],
    [136,137,147,147,149,149,165,166],
    [137,137,148,148,150,150,166,166],
    [137,138,149,149,151,151,166,167],
    [138,138,150,150,152,152,167,167],
    [138,139,151,151,153,153,167,168],
    [139,139,152,152,154,154,168,168],
    [139,140,153,153,155,155,168,169],
    [140,140,154,154,156,156,169,169],
    [140,141,155,155,157,157,169,170],
    [141,141,156,156,158,158,170,170],
    [141,142,157,157,159,159,170,171],
    [142,142,158,158,160,160,171,171],
    [142,143,159,159,161,161,171,172],
    [143,143,160,160,162,162,172,172],
    [143,144,161,161,163,163,172,173],
    [144,144,162,162,162,162,173,173],
    [145,145,146,146,174,174,175,175],
    [146,147,147,148,175,176,176,177],
    [148,149,149,150,177,178,178,179],
    [150,151,151,152,179,180,180,181],
    [152,153,153,154,181,182,182,183],
    [154,155,155,156,183,184,184,185],
    [156,157,157,158,185,186,186,187],
    [158,159,159,160,187,188,188,189],
    [160,161,161,162,189,190,190,191],
    [162,162,163,163,191,191,192,192],
    [164,164,175,175,175,175,193,193],
    [164,165,174,174,176,176,193,194],
    [165,165,175,175,177,177,194,194],
    [165,166,176,176,178,178,194,195],
    [166,166,177,177,179,179,195,195],
    [166,167,178,178,180,180,195,196],
    [167,167,179,179,181,181,196,196],
    [167,168,180,180,182,182,196,197],
    [168,168,181,181,183,183,197,197],
    [168,169,182,182,184,184,197,198],
    [169,169,183,183,185,185,198,198],
    [169,170,184,184,186,186,198,199],
    [170,170,185,185,187,187,199,199],
    [170,171,186,186,188,188,199,200],
    [171,171,187,187,189,189,200,200],
    [171,172,188,188,190,190,200,201],
    [172,172,189,189,191,191,201,201],
    [172,173,190,190,192,192,201,202],
    [173,173,191,191,191,191,202,202],
    [174,174,175,175,203,203,204,204],
    [175,176,176,177,204,205,205,206],
    [177,178,178,179,206,207,207,208],
    [179,180,180,181,208,209,209,210],
    [181,182,182,183,210,211,211,212],
    [183,184,184,185,212,213,213,214],
    [185,186,186,187,214,215,215,216],
    [187,188,188,189,216,217,217,218],
    [189,190,190,191,218,219,219,220],
    [191,191,192,192,220,220,221,221],
    [193,193,204,204,204,204,222,222],
    [193,194,203,203,205,205,222,223],
    [194,194,204,204,206,206,223,223],
    [194,195,205,205,207,207,223,224],
    [195,195,206,206,208,208,224,224],
    [195,196,207,207,209,209,224,225],
    [196,196,208,208,210,210,225,225],
    [196,197,209,209,211,211,225,226],
    [197,197,210,210,212,212,226,226],
    [197,198,211,211,213,213,226,227],
    [198,198,212,212,214,214,227,227],
    [198,199,213,213,215,215,227,228],
    [199,199,214,214,216,216,228,228],
    [199,200,215,215,217,217,228,229],
    [200,200,216,216,218,218,229,229],
    [200,201,217,217,219,219,229,230],
    [201,201,218,218,220,220,230,230],
    [201,202,219,219,221,221,230,231],
    [202,202,220,220,220,220,231,231],
    [203,203,204,204,232,232,233,233],
    [204,205,205,206,233,234,234,235],
    [206,207,207,208,235,236,236,237],
    [208,209,209,210,237,238,238,239],
    [210,211,211,212,239,240,240,241],
    [212,213,213,214,241,242,242,243],
    [214,215,215,216,243,244,244,245],
    [216,217,217,218,245,246,246,247],
    [218,219,219,220,247,248,248,249],
    [220,220,221,221,249,249,250,250],
    [222,222,233,233,233,233,251,251],
    [222,223,232,232,234,234,251,252],
    [223,223,233,233,235,235,252,252],
    [223,224,234,234,236,236,252,253],
    [224,224,235,235,237,237,253,253],
    [224,225,236,236,238,238,253,254],
    [225,225,237,237,239,239,254,254],
    [225,226,238,238,240,240,254,255],
    [226,226,239,239,241,241,255,255],
    [226,227,240,240,242,242,255,256],
    [227,227,241,241,243,243,256,256],
    [227,228,242,242,244,244,256,257],
    [228,228,243,243,245,245,257,257],
    [228,229,244,244,246,246,257,258],
    [229,229,245,245,247,247,258,258],
    [229,230,246,246,248,248,258,259],
    [230,230,247,247,249,249,259,259],
    [230,231,248,248,250,250,259,260],
    [231,231,249,249,249,249,260,260],
    [232,232,233,233,261,261,262,262],
    [233,234,234,235,262,263,263,264],
    [235,236,236,237,264,265,265,266],
    [237,238,238,239,266,267,267,268],
    [239,240,240,241,268,269,269,270],
    [241,242,242,243,270,271,271,272],
    [243,244,244,245,272,273,273,274],
    [245,246,246,247,274,275,275,276],
    [247,248,248,249,276,277,277,278],
    [249,249,250,250,278,278,279,279],
    [251,251,262,262,262,262,280,280],
    [251,252,261,261,263,263,280,281],
    [252,252,262,262,264,264,281,281],
    [252,253,263,263,265,265,281,282],
    [253,253,264,264,266,266,282,282],
    [253,254,265,265,267,267,282,283],
    [254,254,266,266,268,268,283,283],
    [254,255,267,267,269,269,283,284],
    [255,255,268,268,270,270,284,284],
    [255,256,269,269,271,271,284,285],
    [256,256,270,270,272,272,285,285],
    [256,257,271,271,273,273,285,286],
    [257,257,272,272,274,274,286,286],
    [257,258,273,273,275,275,286,287],
    [258,258,274,274,276,276,287,287],
    [258,259,275,275,277,277,287,288],
    [259,259,276,276,278,278,288,288],
    [259,260,277,277,279,279,288,289],
    [260,260,278,278,278,278,289,289],
    [261,261,262,262,290,290,291,291],
    [262,263,263,264,291,292,292,293],
    [264,265,265,266,293,294,294,295],
    [266,267,267,268,295,296,296,297],
    [268,269,269,270,297,298,298,299],
    [270,271,271,272,299,300,300,301],
    [272,273,273,274,301,302,302,303],
    [274,275,275,276,303,304,304,305],
    [276,277,277,278,305,306,306,307],
    [278,278,279,279,307,307,308,308],
    [280,280,291,291,291,291,309,309],
    [280,281,290,290,292,292,309,310],
    [281,281,291,291,293,293,310,310],
    [281,282,292,292,294,294,310,311],
    [282,282,293,293,295,295,311,311],
    [282,283,294,294,296,296,311,312],
    [283,283,295,295,297,297,312,312],
    [283,284,296,296,298,298,312,313],
    [284,284,297,297,299,299,313,313],
    [284,285,298,298,300,300,313,314],
    [285,285,299,299,301,301,314,314],
    [285,286,300,300,302,302,314,315],
    [286,286,301,301,303,303,315,315],
    [286,287,302,302,304,304,315,316],
    [287,287,303,303,305,305,316,316],
    [287,288,304,304,306,306,316,317],
    [288,288,305,305,307,307,317,317],
    [288,289,306,306,308,308,317,318],
    [289,289,307,307,307,307,318,318],
    [290,290,291,291,319,319,320,320],
    [291,292,292,293,320,321,321,322],
    [293,294,294,295,322,323,323,324],
    [295,296,296,297,324,325,325,326],
    [297,298,298,299,326,327,327,328],
    [299,300,300,301,328,329,329,330],
    [301,302,302,303,330,331,331,332],
    [303,304,304,305,332,333,333,334],
    [305,306,306,307,334,335,335,336],
    [307,307,308,308,336,336,337,337],
    [309,309,320,320,320,320,338,338],
    [309,310,319,319,321,321,338,339],
    [310,310,320,320,322,322,339,339],
    [310,311,321,321,323,323,339,340],
    [311,311,322,322,324,324,340,340],
    [311,312,323,323,325,325,340,341],
    [312,312,324,324,326,326,341,341],
    [312,313,325,325,327,327,341,342],
    [313,313,326,326,328,328,342,342],
    [313,314,327,327,329,329,342,343],
    [314,314,328,328,330,330,343,343],
    [314,315,329,329,331,331,343,344],
    [315,315,330,330,332,332,344,344],
    [315,316,331,331,333,333,344,345],
    [316,316,332,332,334,334,345,345],
    [316,317,333,333,335,335,345,346],
    [317,317,334,334,336,336,346,346],
    [317,318,335,335,337,337,346,347],
    [318,318,336,336,336,336,347,347],
    [319,319,320,320,348,348,349,349],
    [320,321,321,322,349,350,350,351],
    [322,323,323,324,351,352,352,353],
    [324,325,325,326,353,354,354,355],
    [326,327,327,328,355,356,356,357],
    [328,329,329,330,357,358,358,359],
    [330,331,331,332,359,360,360,361],
    [332,333,333,334,361,362,362,363],
    [334,335,335,336,363,364,364,365],
    [336,336,337,337,365,365,366,366],
    [338,338,338,338,349,349,349,349],
    [338,338,339,339,348,348,350,350],
    [339,339,339,339,349,349,351,351],
    [339,339,340,340,350,350,352,352],
    [340,340,340,340,351,351,353,353],
    [340,340,341,341,352,352,354,354],
    [341,341,341,341,353,353,355,355],
    [341,341,342,342,354,354,356,356],
    [342,342,342,342,355,355,357,357],
    [342,342,343,343,356,356,358,358],
    [343,343,343,343,357,357,359,359],
    [343,343,344,344,358,358,360,360],
    [344,344,344,344,359,359,361,361],
    [344,344,345,345,360,360,362,362],
    [345,345,345,345,361,361,363,363],
    [345,345,346,346,362,362,364,364],
    [346,346,346,346,363,363,365,365],
    [346,346,347,347,364,364,366,366],
    [347,347,347,347,365,365,365,365]])

In [82]:
f_n[205]

array([194, 194, 204, 204, 206, 206, 223, 223])

In [66]:
p_n = lysis.EdgeGrid.generate_neighborhood_structure(e)
p_n[0]

array([ 1,  1, 30, 30,  2, 31,  2, 31], dtype=uint16)

In [10]:
p_n[1]

array([0, 0, 0, 0, 2, 2, 2, 2], dtype=uint16)

In [14]:
perm = np.array([lysis.from_fortran_edge_index(k, e.macro_params.rows, e.macro_params.cols) for k in range(e.macro_params.total_edges)])
perm = np.ravel_multi_index((perm[:, 0], perm[:, 1]), dims=(e.macro_params.rows, e.macro_params.full_row))

In [50]:
inv_perm = []
for k in range(e.macro_params.rows*e.macro_params.full_row):
    p = np.argwhere(perm == k)
    if p.size > 0:
        inv_perm.append(p[0,0])
    else:
        inv_perm.append(-1)
inv_perm = np.array(inv_perm)
inv_perm

array([ 19,   0,   1,  20,   2,   3,  21,   4,   5,  22,   6,   7,  23,
         8,   9,  24,  10,  11,  25,  12,  13,  26,  14,  15,  27,  16,
        17,  28,  18,  48,  29,  30,  49,  31,  32,  50,  33,  34,  51,
        35,  36,  52,  37,  38,  53,  39,  40,  54,  41,  42,  55,  43,
        44,  56,  45,  46,  57,  47,  77,  58,  59,  78,  60,  61,  79,
        62,  63,  80,  64,  65,  81,  66,  67,  82,  68,  69,  83,  70,
        71,  84,  72,  73,  85,  74,  75,  86,  76, 106,  87,  88, 107,
        89,  90, 108,  91,  92, 109,  93,  94, 110,  95,  96, 111,  97,
        98, 112,  99, 100, 113, 101, 102, 114, 103, 104, 115, 105, 135,
       116, 117, 136, 118, 119, 137, 120, 121, 138, 122, 123, 139, 124,
       125, 140, 126, 127, 141, 128, 129, 142, 130, 131, 143, 132, 133,
       144, 134, 164, 145, 146, 165, 147, 148, 166, 149, 150, 167, 151,
       152, 168, 153, 154, 169, 155, 156, 170, 157, 158, 171, 159, 160,
       172, 161, 162, 173, 163, 193, 174, 175, 194, 176, 177, 19

In [69]:
sort = np.argsort(inv_perm[p_n[perm]], axis=1)

In [54]:
for i in range(e.macro_params.total_edges):
    if Counter(f_n[i]) != Counter(inv_perm[p_n[perm]][i]):
        print(f_n[i], inv_perm[p_n[perm]][i])

In [77]:
new = p_n.copy()
new[perm] = np.take_along_axis(p_n[perm], sort, axis=1)

In [64]:
p_n.shape

(377, 8)

In [70]:
sort

array([[4, 5, 6, ..., 1, 2, 3],
       [4, 5, 6, ..., 2, 1, 3],
       [4, 5, 6, ..., 1, 2, 3],
       ...,
       [0, 1, 2, ..., 5, 6, 7],
       [0, 2, 1, ..., 5, 6, 7],
       [0, 1, 2, ..., 5, 6, 7]])

In [74]:
inv_perm[p_n[perm]]

array([[ 19,  19,  19, ...,   1,   1,   1],
       [ 19,  20,  19, ...,   0,   2,   2],
       [ 20,  20,  20, ...,   1,   3,   3],
       ...,
       [346, 346, 346, ..., 363, 365, 365],
       [346, 347, 346, ..., 364, 366, 366],
       [347, 347, 347, ..., 365, 365, 365]])

In [81]:
new[1], f_n[0]

(array([2, 2, 2, 2, 0, 0, 0, 0], dtype=uint16),
 array([ 1,  1,  1,  1, 19, 19, 19, 19]))

In [1]:
import lysis
import uuid

In [3]:
lysis.util.uuid8code()

'0HUF-xNss-rLWM-yxFR-SrfFRn'

In [17]:
uuid.UUID(int=lysis.util.uuid8())

UUID('018643c0-a2e7-01c4-26f7-7a04d6ca89fb')

In [ ]:
0HUF-woKh-yvRu-Sa33-83X2Zo

In [4]:
p = [{'A': [10, 20, 30], 'B': [5, 15], 'C': [100, 200]}, {'A': 50, 'B': [25, 35], 'C': 300}]

In [5]:
lysis.util.Experiment.split_variations(p)

[{'A': 10, 'B': 5, 'C': 100},
 {'A': 10, 'B': 5, 'C': 200},
 {'A': 10, 'B': 15, 'C': 100},
 {'A': 10, 'B': 15, 'C': 200},
 {'A': 20, 'B': 5, 'C': 100},
 {'A': 20, 'B': 5, 'C': 200},
 {'A': 20, 'B': 15, 'C': 100},
 {'A': 20, 'B': 15, 'C': 200},
 {'A': 30, 'B': 5, 'C': 100},
 {'A': 30, 'B': 5, 'C': 200},
 {'A': 30, 'B': 15, 'C': 100},
 {'A': 30, 'B': 15, 'C': 200},
 {'A': 50, 'B': 25, 'C': 300},
 {'A': 50, 'B': 35, 'C': 300}]